# Fase 1: Entrenamiento

Originalmente, se pretendía que las etapas de hiperafinación y entrenamiento se realizaran dentro del mismo *notebook*, pero rápidamente notamos que al desplegarlos, las instancias simplemente se caían debido al agotamiento de la RAM (OOM). La causa está en que la librería "Keras Tuner" por cada *trial* acumula más y más memoria RAM de manera absurda sin liberarla una vez finalizada la etapa de hiperafinación. Es por esto que la fase 1 se divide en dos etapas.

## Configuración de la GPU

Por un motivo que se desconoce, cuando se utiliza el acelerador P100, es necesario limitar el crecimiento de la GPU (para más detalles, revisar [acá](https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth)). Otro punto es que esta celda debe venir antes de cualquier importación ya que internamente modifican la capacidad de la GPU (ver el siguiente [hilo](https://github.com/hunglc007/tensorflow-yolov4-tflite/issues/171)), y por ende, obtenemos el error `Physical devices cannot be modified after being initialized`.

In [1]:
from tensorflow.config import list_physical_devices
from tensorflow.config.experimental import set_memory_growth

gpus = list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            set_memory_growth(gpu, True)
    except RuntimeError as e:
        print("error:", e)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

## Instalación de dependencias

Esta imagen viene con librerías preinstaladas de Tensorflow y Hugging Face, por lo que únicamente deberemos instalar `dotwiz`, el cual permite transformar un diccionario a una notación de puntos (tal y como accedemos a métodos y atributos en Javascript).

In [2]:
%%capture
!pip install dotwiz

Cambiaremos el nivel de *logs* generados por Hugging Face ya que genera advertencias que no aplican en nuestro caso.

In [3]:
from transformers import logging

logging.set_verbosity_error()

## Inicialización de constantes

Definiremos al objeto `CONSTANTS` que contendrá todas las constantes a utilizar organizadas de forma jerárquica.

In [4]:
from dotwiz import DotWiz
from math import log, floor
from logging import info
from re import compile

CONSTANTS = DotWiz({
    "DATASETS": {
        "PAN": {
            "PATHS": {
                "TRAINING": {
                    "CONVERSATIONS":
                    "/kaggle/input/pan-2012-training/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml",
                    "SEXUAL_PREDATORS":
                    "/kaggle/input/pan-2012-training/pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt"
                },
                "TESTING": {
                    "CONVERSATIONS":
                    "/kaggle/input/pan-2012-testing/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml",
                    "SEXUAL_PREDATORS":
                    "/kaggle/input/pan-2012-testing/pan12-sexual-predator-identification-groundtruth-problem1.txt",
                    "SUSPICIOUS_CONVERSATIONS":
                    "/kaggle/input/pan-2012-testing/pan12-sexual-predator-identification-groundtruth-problem2.txt"
                }
            }
        }
    },
    "MODELS": {
        "BERT": {
            "NAME": "bert-base-uncased",
            "TOKENIZER": {
                "MAX_LENGTH": 128,
                "PADDING": "max_length",
                "TRUNCATION": True
            },
            "BATCHING": {
                "SIZE": 32
            },
        }
    },
    "PREPROCESSING": {
        "RANDOMNESS": {
            "SEED": 400
        },
        "MESSAGE_BATCHING": {
            "SIZE": 24
        },
        "REGEXES": {
            "XML_QUOTE_ESCAPES": compile(r"&(apos|quot);")
        },
        "DATASET_SPLITS": {
            "TRAINING": 0.8,
            "VALIDATION": 0.2
        }
    },
    "TRAINING": {
        "SCHEDULES": {
            "EXPONENTIAL_DECAY": {
                "DECAY_STEPS_FACTOR": 1 / 2,
                "DECAY_RATE": 0.97
            },
            "ADAM_WEIGHT_DECAY": {
                "BETA_1": 0.9,
                "BETA_2": 0.999,
                "WEIGHT_DECAY_RATE": 0.01
            }
        },
        "EPOCHS": 1,
        "WARMUP_FACTOR": 0.1,
        "INITIAL_LEARNING_RATE": 3e-5,
    }
})

Para poder recrear (hasta cierto punto) cada uno de los resultados propuestos, preestableceremos una semilla. Es posible que existan diferencias entre una ejecución y otra ya que como trabajaremos a nivel de GPU, muchas de las operaciones en Tensorflow son procesadas de manera asíncrona, y muchos de los valores que tratamos acá requieren sumar flotantes que sí se ven afectados cuando cambian sus órdenes. Si quisiéramos habilitar un determinismo completo, usaríamos la instrucción `tensorflow.config.experimental.enable_op_determinism()`, pero veríamos una degradación en el desempeño de las instrucciones en varios órdenes de magnitud. Para mayores detalles, revisar la [documentación oficial](https://www.tensorflow.org/versions/r2.8/api_docs/python/tf/config/experimental/enable_op_determinism) de Tensorflow.

In [5]:
from tensorflow.keras.utils import set_random_seed

set_random_seed(CONSTANTS.PREPROCESSING.RANDOMNESS.SEED)

## Generación del conjunto de entrenamiento y validación

El conjunto de datos de **entrenamiento** PAN2012 tiene dos archivos de utilidad:

1. `pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt`: Enlista todos los identificadores de los autores que (se sabe) son depredadores sexuales separados por saltos de línea.
1. `pan12-sexual-predator-identification-training-corpus-2012-05-01.xml`: Enlista tanto conversaciones normales como pervertidas en un formato de etiquetas. Cada conversación se encierra con `<conversation>` y cada mensaje por `<message>`.

La cantidad de conversaciones normales versus pervertidas está altamente desequilibrada, por lo que se hará un tratamiento básico.

In [6]:
from xml.etree import cElementTree as ET
from xml.sax.saxutils import unescape
from re import sub
from pandas import Series
from datasets import Dataset, concatenate_datasets
from transformers import DataCollatorWithPadding


def parse_xml_to_hf_dataset(sexual_predators_path, conversations_path):
    dataset = {
        "conversation_id": [],
        "conversation_label": [],
        "message": [],
    }

    with open(sexual_predators_path, "r") as file:
        sexual_predators = []

        for sexual_predator in file.readlines():
            sexual_predator = sexual_predator.strip()
            sexual_predators.append(sexual_predator)

    for event, element in ET.iterparse(conversations_path,
                                       events=("start", "end")):
        if event != "end" or element.tag != "conversation":
            continue

        conversation_id = element.get("id").strip()
        messages = element.findall("message")
        unescaped_messages = []

        conversation_includes_sexual_predator = False

        for index, message in enumerate(messages):
            author = message.find("author").text.strip()
            message = message.find("text")

            if message is None:
                continue

            if message.text is None:
                continue

            if author in sexual_predators:
                conversation_includes_sexual_predator = True

            unescaped_message = sub(
                CONSTANTS.PREPROCESSING.REGEXES.XML_QUOTE_ESCAPES, "'",
                unescape(message.text.strip()))
            unescaped_messages.append(unescaped_message)

        if not unescaped_messages:
            continue

        for index in range(0, len(unescaped_messages),
                           CONSTANTS.PREPROCESSING.MESSAGE_BATCHING.SIZE):
            dataset["conversation_id"].append(conversation_id)
            dataset["conversation_label"].append(
                conversation_includes_sexual_predator)
            dataset["message"].append("\n".join(
                unescaped_messages[index:index + CONSTANTS.PREPROCESSING.
                                   MESSAGE_BATCHING.SIZE]))

    conversation_series = Series(dataset["conversation_label"])
    normal_conversation_series = conversation_series[conversation_series ==
                                                     False]
    perverted_conversation_series = conversation_series[conversation_series ==
                                                        True]
    normal_conversation_series = normal_conversation_series.sample(
        frac=len(perverted_conversation_series) /
        len(normal_conversation_series),
        random_state=CONSTANTS.PREPROCESSING.RANDOMNESS.SEED)

    hf_dataset = Dataset.from_dict(dataset)
    hf_dataset = hf_dataset.select(
        perverted_conversation_series.index.to_list() +
        normal_conversation_series.index.to_list())

    return hf_dataset

Inicializaremos un objeto llamado `tokenizer` que permitirá dividir sentencias en subpalabras (*tokens*) para asociarlas con un identificador (*token ids*) dentro de un vocabulario. Es una instancia específica para el modelo BERT llamado `CONSTANTS.MODELS.BERT.NAME` de Hugging Face. Su salida será diccionario con tres claves:

- `input_ids`: Contendrá una matriz con los identificadores de las palabras divididas.
- `token_type_ids`: Contendrá una matriz de correspondencia de las  palabras divididas a alguna de las frases en la tupla. Como nuestro caso no es QA, esta tupla en realidad no existe, y lo único que veremos como salida será un vector de elementos `0`.
- `attention_mask`: Contendrá una matriz que considerará o no a las palabras divididas en el procesamiento. 

In [7]:
%%capture

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(CONSTANTS.MODELS.BERT.NAME)

Crearemos una función que aplique el algoritmo Subword Tokenization a través del objeto `tokenizer` en cada uno de los mensajes. El resultado deberá ser retornado en dos conjuntos: uno de tamaño  `left_side_proportion` y otro de tamaño `1 - left_side_proportion`.

In [8]:
from transformers import DataCollatorWithPadding


def hf_dataset_to_tf_dataset(hf_dataset,
                             left_side_proportion=None,
                             return_hf=False):
    hf_dataset = hf_dataset.map(lambda example: tokenizer(
        example["message"],
        max_length=CONSTANTS.MODELS.BERT.TOKENIZER.MAX_LENGTH,
        padding=CONSTANTS.MODELS.BERT.TOKENIZER.PADDING,
        truncation=CONSTANTS.MODELS.BERT.TOKENIZER.TRUNCATION))
    hf_dataset = hf_dataset.shuffle(
        seed=CONSTANTS.PREPROCESSING.RANDOMNESS.SEED)

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer,
                                            return_tensors="tf")
    to_tf_dataset_kwargs = {
        "columns": ["input_ids", "token_type_ids", "attention_mask"],
        "label_cols": ["conversation_label"],
        "batch_size": CONSTANTS.MODELS.BERT.BATCHING.SIZE,
        "collate_fn": data_collator,
        "shuffle": False
    }

    if left_side_proportion is None:
        if return_hf:
            return (hf_dataset.to_tf_dataset(**to_tf_dataset_kwargs),
                    hf_dataset), (None, None)

        return (hf_dataset.to_tf_dataset(**to_tf_dataset_kwargs), None), (None,
                                                                          None)

    hf_dataset = hf_dataset.train_test_split(train_size=left_side_proportion,
                                             shuffle=None)
    tf_left_side_dataset = hf_dataset["train"].to_tf_dataset(
        **to_tf_dataset_kwargs)
    tf_right_side_dataset = hf_dataset["test"].to_tf_dataset(
        **to_tf_dataset_kwargs)

    if return_hf:
        return (tf_left_side_dataset,
                hf_dataset["train"]), (tf_right_side_dataset,
                                       hf_dataset["test"])

    return (tf_left_side_dataset, None), (tf_right_side_dataset, None)

Produciremos el conjunto de entrenamiento y validación.

In [9]:
hf_dataset = parse_xml_to_hf_dataset(
    CONSTANTS.DATASETS.PAN.PATHS.TRAINING.SEXUAL_PREDATORS,
    CONSTANTS.DATASETS.PAN.PATHS.TRAINING.CONVERSATIONS)

(tf_training_dataset,
 hf_training_dataset), (tf_validation_dataset,
                        hf_validation_dataset) = hf_dataset_to_tf_dataset(
                            hf_dataset,
                            left_side_proportion=1 -
                            CONSTANTS.PREPROCESSING.DATASET_SPLITS.VALIDATION,
                            return_hf=True)

  0%|          | 0/9450 [00:00<?, ?ex/s]

## Construcción del modelo

Recuperaremos el modelo pre-entrenado de BERT y lo compilaremos para ajustarlo a los datos.

In [10]:
from transformers import AdamWeightDecay, WarmUp, TFBertForSequenceClassification
from tensorflow.data.experimental import cardinality
from tensorflow.keras.optimizers.schedules import ExponentialDecay

STEPS_PER_EPOCH = cardinality(tf_training_dataset).numpy()
TOTAL_TRAINING_STEPS = STEPS_PER_EPOCH * CONSTANTS.TRAINING.EPOCHS
TOTAL_WARMUP_STEPS = int(CONSTANTS.TRAINING.WARMUP_FACTOR *
                         TOTAL_TRAINING_STEPS)

learning_rate_schedule = ExponentialDecay(
    initial_learning_rate=CONSTANTS.TRAINING.INITIAL_LEARNING_RATE,
    decay_steps=STEPS_PER_EPOCH *
    CONSTANTS.TRAINING.SCHEDULES.EXPONENTIAL_DECAY.DECAY_STEPS_FACTOR,
    decay_rate=CONSTANTS.TRAINING.SCHEDULES.EXPONENTIAL_DECAY.DECAY_RATE)
warmup_schedule = WarmUp(
    initial_learning_rate=CONSTANTS.TRAINING.INITIAL_LEARNING_RATE,
    decay_schedule_fn=learning_rate_schedule,
    warmup_steps=TOTAL_WARMUP_STEPS)
optimizer = AdamWeightDecay(
    learning_rate=warmup_schedule,
    beta_1=CONSTANTS.TRAINING.SCHEDULES.ADAM_WEIGHT_DECAY.BETA_1,
    beta_2=CONSTANTS.TRAINING.SCHEDULES.ADAM_WEIGHT_DECAY.BETA_2,
    weight_decay_rate=CONSTANTS.TRAINING.SCHEDULES.ADAM_WEIGHT_DECAY.
    WEIGHT_DECAY_RATE)
metrics = ["accuracy"]

model = TFBertForSequenceClassification.from_pretrained(
    CONSTANTS.MODELS.BERT.NAME, num_labels=2)
model.compile(optimizer=optimizer, metrics=metrics)

## Entrenamiento del modelo hiperafinado

Todos los modelos serán entrenados bajo la misma cantidad de *epochs* para efectos de comparación.

In [11]:
history = model.fit(tf_training_dataset,
                    validation_data=tf_validation_dataset,
                    epochs=CONSTANTS.TRAINING.EPOCHS,
                    verbose=1)

237/237 [==============================] - 151s 536ms/step - loss: 0.2107 - accuracy: 0.9091 - val_loss: 0.1253 - val_accuracy: 0.9503


## Creación del conjunto de pruebas

En cuanto al directorio no existen muchas diferencias. Los archivos que nos interesan son los siguientes:

1. `pan12-sexual-predator-identification-groundtruth-problem1.txt`: Enlista todos los identificadores de los autores que (se sabe) son depredadores sexuales separados por saltos de línea.
1. `pan12-sexual-predator-identification-test-corpus-2012-05-17.xml`: Enlista tanto conversaciones normales como pervertidas en un formato de etiquetas. Cada conversación se encierra con `<conversation>` y cada mensaje por `<message>`.

Al igual que para el caso del entrenamiento, la cantidad de conversaciones normales versus pervertidas está altamente desequilibrada, por lo que se hará un tratamiento básico.

In [12]:
hf_dataset = parse_xml_to_hf_dataset(
    CONSTANTS.DATASETS.PAN.PATHS.TESTING.SEXUAL_PREDATORS,
    CONSTANTS.DATASETS.PAN.PATHS.TESTING.CONVERSATIONS)
(tf_testing_dataset,
 hf_testing_dataset), (_,
                       _) = hf_dataset_to_tf_dataset(hf_dataset,
                                                     left_side_proportion=None,
                                                     return_hf=True)

  0%|          | 0/15782 [00:00<?, ?ex/s]

## Estimación de métricas en base al conjunto de pruebas

Dado que los conjuntos que trabajamos son instancias de la clase padre `tensorflow.data.Dataset`, no podemos acceder directamente a las etiquetas en un formato *slicing* (o similar), por lo que deberemos recorrer *batch* por *batch* y concatenar los resultados en un vector. 

In [13]:
from numpy import array, concatenate


def get_tf_dataset_labels(tf_dataset):
    concatenated_labels = array([], dtype="int32")

    for _, labels in tf_dataset:
        concatenated_labels = concatenate((concatenated_labels, labels))

    return concatenated_labels

Para medir la capacidad de generalización del modelo recién entrenado, definiremos la función `mesaure_model_metrics`, la cual retornará cinco valores (en orden):

1. Exactitud o *binary accuracy*.
1. Pérdida o *binary crossentropy*.
1. Recuperación o *recall*.
1. Precisión o *precision*.
1. Valor-F o *F-score*.

In [14]:
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.losses import BinaryCrossentropy


def measure_model_metrics(true_labels, predicted_labels, from_logits=False):
    binary_accuracy = BinaryAccuracy()
    binary_accuracy.update_state(true_labels, predicted_labels)

    binary_crossentropy = BinaryCrossentropy(from_logits=from_logits)(
        true_labels, predicted_labels)

    recall = Recall()
    recall.update_state(true_labels, predicted_labels)

    precision = Precision()
    precision.update_state(true_labels, predicted_labels)

    f_score = 2 * (precision.result().numpy() * recall.result().numpy()) / (
        precision.result().numpy() + recall.result().numpy())

    return binary_accuracy.result().numpy(), binary_crossentropy.numpy(
    ), recall.result().numpy(), precision.result().numpy(), f_score

Finalmente, obtendremos las métricas de desempeño.

In [15]:
from numpy import array
from tensorflow import sigmoid

outputs = model.predict(tf_testing_dataset, verbose=1)
true_labels = get_tf_dataset_labels(tf_testing_dataset)
predicted_labels = sigmoid(outputs.logits).numpy()[:,1].flatten()
accuracy, loss, recall, precision, f_score = measure_model_metrics(
    true_labels, predicted_labels, from_logits=False)

print(f"\n- accuracy: {accuracy}")
print(f"- loss: {loss}")
print(f"- recall: {recall}")
print(f"- precision: {precision}")
print(f"- f-score: {f_score}")

494/494 [==============================] - 87s 171ms/step

- accuracy: 0.954124927520752
- loss: 0.19162359833717346
- recall: 0.9367634057998657
- precision: 0.970460832118988
- f-score: 0.9533144537681084


## Guardar resultados

Persistiremos el modelo y todos los conjuntos de datos.

In [16]:
from tensorflow.data.experimental import save

model.save_pretrained("model")

save(tf_training_dataset, "tf/training/training")
save(tf_validation_dataset, "tf/training/validation")
save(tf_testing_dataset, "tf/testing/testing")

hf_training_dataset.save_to_disk("hf/training/training")
hf_validation_dataset.save_to_disk("hf/training/validation")
hf_testing_dataset.save_to_disk("hf/testing/testing")

Flattening the indices:   0%|          | 0/8 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/2 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/16 [00:00<?, ?ba/s]